# Setting up location recommendations for a Japanese franchise company based of a data driven analytical research on the similarity of neighborhoods in the capital cities Berlin, London and Paris. 

                                        Maximilian Jaschkowitz, M.Sc.
                             
                                              February 18, 2021

### 1. Business Problem 

 

As a new Japanese franchise company needs recommendations for their set up with three new high-priced sushi and Japanese food restaurants, they came up with the question which places over the cities of Berlin, Paris and London would be the best to locate their restaurants. 

In order to give them the best recommendation for the new locations in every city, the first question to be answered is: What is the right quarter for a high-priced Japanese restaurant? 

For the evaluation of this question, we take a few assumptions first: 

1. As the restaurant will be high-priced, we need to look for quarters with other high-priced restaurants to set up the new location. 

2. Even if one says that competition stimulates business, it is much more likely that a new restaurant will get the most customers if there is less choice or less competitors in the same business area. Therefore, we search for quarters in these cities which are not already dominated by Japanese Food. 

So, we need to find locations where high-priced restaurants are accepted and which are not dominated by Japanese like restaurants to maximize the profitability of the new franchise restaurants.  

Therefore, we search for differences and similarities between the neighborhoods of the cities, based on the above-described criteria.


### 2. Data sources used for the analysis of potential set Up locations in Berlin, Paris and London

First of all, we need to collect data about the potential quarters/neighborhoods in every city. Therefore it is elementary to get the spatial coordinates of every neighborhood. 

A first hint to the coordinates of a neighborhood can be given by its postal code. Later we can search the surroundings of these postal codes via a geocoder library which gives us longitudinal and latitudinal coordinates for further inspections. 

With these coordinates it is possible to get informations about the location from 'foursquare - city guide'. Data can be provided via an API connection.

The first data source will lead to a list of postal codes in Berlin. It is provided by the website 'plz-suche.org' which stores each postal code of every city in Germany.
It is reachable by the following url:

https://www.plz-suche.org/stadt.berlin-de13f3

To get the postal codes of the neighborhoods in Paris the website 'annuaire-administration' can be used via the URL:

https://www.annuaire-administration.com/code-postal/departement/paris.html3

For London, by accessing the following url:

https://data.london.gov.uk/dataset/london-borough-profiles#:~:text=The%20London%20boroughs%20are%3A%20City,%2C%20Lewisham%2C%20Merton%2C%20Newham%2C

it is possible to download a csv file which contains 39 entries with postal codes for all wards (neighborhoods) of London. Additionally the file contains statistics about the neighborhoods. and the total statistics about Inner London, Outer London, London, England and the United Kingdom which are not useful for further inspectations. 

After the upload of the data into JupyterLab, we delete the last rows which contains the unnecessary informations.
Additionally, we delete all but two columns in the generated dataframe. This are the name and the postal code of the neighborhoods. 

With these data we can proceed the analyzation. The postal codes are not assigned to any coordinates, therefore it is essential to use a geocoder library to access the coordinates assigned to the postal codes. The geocoder library called 'geopy' gives us this possibility. It is reachable via the following URL.

https://geopy.readthedocs.io/en/stable/

Further, we need to collect data about the area around the neighborhood. Therefore we can use the Foursquare API which leads to informations about venues in the nearer surroundings. We will collect only informations about the location, the pricing and the kind of a venue. High priced venues will get weighted more than low priced venues, dependant on the maximum pricing in the whole city. This procedure makes it possible to determine if we are located in a high priced or a low priced neighborhood. 

The foursquare website is publicly accessible by the following URL. 

https://de.foursquare.com/city-guide

For open access to the data behind the website by a pyhton code, we need the API and a developper account with authorized credentials.






### 3. Methodology

For anwsering the question of the franchise company, where to locate their new restaurants, we need to examine the locations.

With our scraped datasets, we now gain access to data about the locations and the existing venues per neighborhood. But we still don't have any informations about areas with high-priced venues or areas with japanese restaurants.

These informations can be extracted from our given datasets by machine learning algorithms. Because we have no actual informations about clusters of high priced neighborhoods, we are using unlabeled data and search for clusters in the venues of the foursquare data per neighborhood.

Clustering is an unsupervised learning method and the easiest way to gain information is the KMeans algorithm. This algorithm searches for similar samples in a dataset by measuring the dissimilarity between datapoints using the distance between these points.

Befor we use the KMeans algorithm, we will cleanse the data. We will use only restaurants as venues and we will separate them by the different price categories.

This method leads to different classifications for the count of different price categories in the area.

Interesting neighborhoods are only these, where a higher number of high priced restaurants occure.


#### 3.1 Data Cleaning

After adding the coordinates to the data they need to be cleaned and prepared for further analysis. Each step is done three times, once for every city.
It may occur that some data ar missing or in other cases, some data occur doubled in a dataframe. These false data need to be deleted from the dataset to prevent the prediction from false statements due to bias.

The dataset for Berlin listed 6 neighborhoods twice. These neighborhoods were determined and deleted during the data analysation process so that the following counts of neighborhoods remained:


|Berlin|Paris|London
--|--|--|--
Total No. of Neighborhoods|90|20|33

The next step was the request of the nearby venues provided by foursquare around the neighborhoods for every city. 
The first request yields to the venue-names and -categories. We search for venues in a radius of 500 m around the center of the neighborhood. The maximum count of venues is 100. 

The results of this request are shown in the table below:

|Berlin|Paris|London
--|--|--|--
Total No. of Venues|1077|1440|1324
Total No. of Venue-Categories|227|222|229
Mean No. of Venue-Categories per Neighborhood|12|72|41

Berlin has smaller neighborhoods and therefore, less categories per neighborhood. The total number of categories is similar in each city. The total number of venues is different, Berlin has the lowest number, followed by London and then Paris.

#### 3.2 Feature selection

These venues contain more than just restaurants. But the question is focused on restaurants and we dont need the other venue categories. Therefore, the next cleaning step is to search for venue-categories which are restaurants. Additionally, because we focus on Japanese restaurants, we summarize all japanese -like categories to one new categorie calle 'Japanese-Like'.

The following table shows the count of restaurants in every city:

|Berlin|Paris|London
--|--|--|--
Total No. of Restaurants|200|527|278
Total No. of Restaurant-Categories|36|64|51


Interesting is, that Paris seems to have more restaurants than the other two cities. This could be due to the french people who put more emphasis on restaurant culture, but it could also be influenced by diffenrent foursquare activites in the three cities. Paris is well known as city of Love in USA and Canada, so this is a city with a lot more of tourists than Berlin or London.

Now we need to know the price categories of the restaurants.

The next cleaning step is caused by the limit of Detail API calls which are restricted by my account (500 calls per day).
Because of the need to decrease the number of restaurants, I choose criteria to cut the data. 

The plotting of the count of restaurants per neighborhood against the distance of the neighborhood to the inner city shows which distance is relevant to choose as cut-off criteria to have neighborhoods with a sufficiently amount of restaurants.
The following plot shows this method for the city of Berlin:
![alt text](distance_restaurants_berlin.png "Distance-Restaurants_Berlin")

With this method the result for Berlin was a circle of 10 km around the inner city to access all relevant restaurants.
The following table shows the results for all cities:

|Berlin|Paris|London
--|--|--|--
Radius around the Inner City|10|2|10
Number of restaurants in this radius|126|188|152

The sum of the total number of restaurants in all cities is now 466 and the foursquare-API call is now possible because it is smaller than 500 (calls per day).

Not every restaurant has a value for the price category and therefore, the total number of useful restaurants for our classification decreases again. 48 restaurants have missing price categories. The following table shows the total number of useful restaurants in the cities. 

|Berlin|Paris|London
--|--|--|--
Number of useful Restaurants|113|169|136

In Total we have now 418 useful restaurants with their price categories.
Grouping and sorting yields to the 10 most common restaurants per neighborhood in the corresponding price category.
After doing this for all three cities, the next step is the Clustering process with KMeans.






### 4. Results

The clustering process was repeated for different number of clusters. The result was that the most meaningful interpretations could be done with the following number of clusters:

|Berlin|Paris|London
----|---|--|--
Number of clusters|6|4|5

The next step was the interpretation of the results for every city depending on the first, second and third most often restaurants in the clustered neighborhoods. 

These interpretation yields to the following table of interpretations:

|Berlin|Paris|London
-----|-----|-----|-----
Cluster 0|Japanese|French|Mixed/Japanese
Cluster 1|Italian|Japanese|Fast Food
Cluster 2|German|Mexican|Falafel
Cluster 3|Fast Food|Falafel|Japanese
Cluster 4|Unspecified restaurants||Tapas
Cluster 5|Mixed restaurants||

The interpretations of the results of the clustering process were then be added to the dataframes and plotted on a map. I separate the results by the price category, so that there is one map for every price category in each city.

In total we have 10 maps divided over 3 cities. This looks like the following table shows:

Price-Category|Berlin|Paris|London
------|-----|------|-
Cheap|1 Map|1 Map|1 Map
Moderate| 1 Map| 1 Map| 1 Map
Expensive|1 Map|1 Map|1 Map
Very Expensive|-|1 Map|-

As the most important maps are the ones of the expensive/ very expensive neighborhood categories, i will go in detail for these results here.

The following picture shows exemplary the moderate restaurants of berlin:

#### Moderate restaurants, Berlin:

![title](Maps/map_berlin_moderate.png)

The map shows that the cluster No. 5, interpreted as mixed sort of restaurants but not japanese, cluster No. 1, interpreted as Italian restaurants and cluster No. 2, interpreted as German restaurants are the most common restaurants in the price category moderate.

#### Expensive restaurants, Berlin:

![title](Maps/maps_berlin_expensive.png)

This map shows just one neighborhood with expensive restaurants in it. A click on the circle uncovers that after the unspecified restaurants the most common restaurants are Japanese and Turkish restaurants.

#### Expensive restaurants, Paris:

![title](Maps/maps_paris_expensive.png)

This map shows six neighborhoods, the 1st,2nd,3rd,4th.,5th and 6th Arrondissements which all belong to one cluster, No. 0 interpreted as French restaurants. Most common are French restaurants, second are seafood and Japanese restaurants.

#### Very expensive restaurants, Paris:

![title](Maps/map_paris_very_expensive.png)

This map shows just one neighborhood (1st Arrondissement) which belongs also to cluster No. 0 with mainly French restaurants. 

#### Expensive restaurants, London:

![title](Maps/map_london_expensive.png)

This map shows six neighborhoods, 5 belonging to the cluster No. 0 which was interpreted as a mix of Japanese and other restaurants. One, Hammersmith and Fulham, belongs to cluster No. 4, interpreted as mainly Tapas restaurants. The closer look at the 5 other neighborhoods extracts, that the City of London, Southwark, Westminster and Lambeth have French, English or Italian restaurants as main restaurants.

The best locations for a new high-priced Japanese restaurant are in these neighborhoods where the price category is expensive or very expensive and where most restaurants are not Japanese. These neighborhoods are listed in the following table:

Berlin|Paris|London
------|-----|------
Wilmersdorf|1ere Arrondissement|City of London
|2eme Arrondissement|Southwark
|3eme Arrondissement|Westminster
|4eme Arrondissement|Hammersmith & Fulham
|5eme Arrondissement|Lambeth
|6eme Arrondissement|


### 5. Discussion

The first and most remarkable, critical point is that the data from foursquare contains restaurants which are not specified to a certain kind of restaurant which means, that these restaurants can be Japanese as well. 

As the results are based on a social network which is not very spread in Europe, the results can be slightly biased towards the preferences of US-Americans and Candians. This means that the results show more restaurants that US-Americans and Canadians prefer. 
A solution could be to include other social networks to the workflow. These social networks should be used by equally distributed people from all over the world. 

Another point to discuss is, that clearly not all restaurants of the cities are catched by the database of foursquare network. The neighborhoods may contain Japanese restaurants even if these restaurants are not included in the database of foursquare which means that the cluster shows that the neighborhood has no or less Japanese restaurant but it could have even more Japanese restaurants than other ones. This error could be solved by using a database which is more spreaded in Europe. 

An algorithm intern error is, that KMeans is building new clusters with every run. This means, that the clusters can slighlty change and there is a small inconsistency. Therefore, the interpretation can be difficult.

In total I recommend to improve the database and to use more than just one social network for the evaluation. One possibility could be, to use the overall income of the employees and the residents of the neighborhoods. With these additional informations and informations about more restaurants, a more precise image of the neighborhood and the restaurants could be done. 


### 6. Conclusion

This report describes the evaluation of restaurant data from the three cities Berlin, Paris and London. The aim is, to decline the best location for a new high priced Japanese restaurant in every city.
The data is seperated into specific data for every neighborhood. After accessing the location data for the neighborhoods, data from the social network foursquare are added to each neighborhood. The data is filtered to restaurants and the price category for each restaurant is then added to these data. 

In a last step, the KMeans algorithm is applied to generate classes based on the frequency of different restaurant categories for every neighborhood in every price category. These results are interpreted and then plotted on a map. 

This map shows the most common restaurants for every price category. Based on these findings, the locations with the highest price category and the lowest frequency of Japanese restaurants are chosen as best location for the new high-priced Japanese restaurant.

